<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#env-details" data-toc-modified-id="env-details-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>env details</a></span></li><li><span><a href="#data-exploration" data-toc-modified-id="data-exploration-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>data exploration</a></span></li><li><span><a href="#data-mapping-from-different-angles" data-toc-modified-id="data-mapping-from-different-angles-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>data mapping from different angles</a></span></li></ul></div>

# env details
python pkg for parameter simulation: scipy (1.4.1), numpy (1.18.2)  
Julia-lang check pkg version: using Pkg;Pkg.installed()  
RCall (0.13.4), DataFrames (0.20.2), CSV (0.6.1), Plots (0.29.7)

In [1]:
using RCall, DataFrames, CSV, Plots

# data exploration

In [2]:
#rAw = CSV.read("../result/maxYield_0.1.csv")
rAw = CSV.read("../result/maxYield_0.05.csv")
describe(rAw)

,variable,mean,min,median,max,nunique,nmissing,eltype
,Symbol,Float64,Float64,Float64,Float64,Nothing,Nothing,DataType
1,x,0.466182,0.0,0.45,1.0,,,Float64
2,e_PR,0.563,0.563,0.563,0.563,,,Float64
3,e_P,0.63,0.63,0.63,0.63,,,Float64
4,g_P,0.41827,0.259,0.4373,0.556,,,Float64
5,a_P,0.184238,0.001,0.1606,0.4,,,Float64
6,e_BR,0.6,0.6,0.6,0.6,,,Float64
7,e_B,0.55,0.55,0.55,0.55,,,Float64
8,g_B,5.00411,1.046,5.1,8.414,,,Float64
9,m_B,0.14,0.14,0.14,0.14,,,Float64


In [3]:
@rput rAw
R"
u0 = vector(mode='list', length=9)
for(i in 1:9){
    u0[[i]] = unique(rAw[,i])
};rm(i)
"
@rget u0

9-element Array{Any,1}:
  [0.0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09  …  0.91, 0.92, 0.93, 0.94, 0.95, 0.96, 0.97, 0.98, 0.99, 1.0]              
 0.563                                                                                                                                   
 0.63                                                                                                                                    
  [0.259, 0.2888, 0.3184, 0.3481, 0.378, 0.4075, 0.4373, 0.467, 0.4966, 0.5264, 0.556]                                                   
  [0.001, 0.0409, 0.0808, 0.1207, 0.1606, 0.2004, 0.2404, 0.2803, 0.3203, 0.36, 0.4]                                                     
 0.6                                                                                                                                     
 0.55                                                                                                                                    
  [1.046, 

# data mapping from different angles

In [4]:
aXis = [4 8 0 5] ## default x,y,z,t axis
fiX = [u0[i] for i in 1:length(u0) if length(u0[i])==1]
aMt = [[i0 i1] for i0 in 1:length(u0[aXis[4]]) for i1 in 1:length(u0[1])]
println()

In [ ]:
aXis[3] = 10 ## customed z axis
aa = @animate for i0 in 1:length(aMt)
                aMt0 = aMt[i0]
                @rput aXis aMt0
                R"
                ## extract necessary data
                pp = rAw[which(rAw[,aXis[4]]==u0[[aXis[4]]][aMt0[1]] & rAw[,1]==u0[[1]][aMt0[2]]),]
                
                ## initialize matrix
                mmR = unique(pp[,aXis[1]])
                mmC = unique(pp[,aXis[2]])
                mm = matrix(NA, nr=length(mmR), nc=length(mmC))
                
                ## map data from dataframe to matrix
                for(i in 1:nrow(pp)){
                    mm[which(mmR==pp[i,aXis[1]]),which(mmC==pp[i,aXis[2]])] <- pp[i,aXis[3]]
                };rm(i)
                "
                @rget mm mmR mmC
                if size(mm)[1] > 0
                    plot(mmR, mmC, mm, st=:surface,
                    xlabel=names(rAw)[aXis[1]], ylabel=names(rAw)[aXis[2]], zlabel=names(rAw)[aXis[3]],
                    title="Surface plot animation based on fixed parameters\n ePR="*string(fiX[1])*", eP="*string(fiX[2])*", eBR="*string(fiX[3])*", eB="*string(fiX[4])*", mB="*string(fiX[5])*",\n x="*string(u0[1][aMt0[2]])*", aP="*string(u0[aXis[4]][aMt0[1]]))
                end
            end
gif(aa, "gif/"*string(names(rAw)[aXis[3]])*".gif", fps=15)

In [ ]:
aXis[3] = 11 ## customed z axis
aa = @animate for i0 in 1:length(aMt)
                aMt0 = aMt[i0]
                @rput aXis aMt0
                R"
                ## extract necessary data
                pp = rAw[which(rAw[,aXis[4]]==u0[[aXis[4]]][aMt0[1]] & rAw[,1]==u0[[1]][aMt0[2]]),]
                
                ## initialize matrix
                mmR = unique(pp[,aXis[1]])
                mmC = unique(pp[,aXis[2]])
                mm = matrix(NA, nr=length(mmR), nc=length(mmC))
                
                ## map data from dataframe to matrix
                for(i in 1:nrow(pp)){
                    mm[which(mmR==pp[i,aXis[1]]),which(mmC==pp[i,aXis[2]])] <- pp[i,aXis[3]]
                };rm(i)
                "
                @rget mm mmR mmC
                if size(mm)[1] > 0
                    plot(mmR, mmC, mm, st=:surface,
                    xlabel=names(rAw)[aXis[1]], ylabel=names(rAw)[aXis[2]], zlabel=names(rAw)[aXis[3]],
                    title="Surface plot animation based on fixed parameters\n ePR="*string(fiX[1])*", eP="*string(fiX[2])*", eBR="*string(fiX[3])*", eB="*string(fiX[4])*", mB="*string(fiX[5])*",\n x="*string(u0[1][aMt0[2]])*", aP="*string(u0[aXis[4]][aMt0[1]]))
                end
            end
gif(aa, "gif/"*string(names(rAw)[aXis[3]])*".gif", fps=15)

In [ ]:
aXis[3] = 12 ## customed z axis
aa = @animate for i0 in 1:length(aMt)
                aMt0 = aMt[i0]
                @rput aXis aMt0
                R"
                ## extract necessary data
                pp = rAw[which(rAw[,aXis[4]]==u0[[aXis[4]]][aMt0[1]] & rAw[,1]==u0[[1]][aMt0[2]]),]
                
                ## initialize matrix
                mmR = unique(pp[,aXis[1]])
                mmC = unique(pp[,aXis[2]])
                mm = matrix(NA, nr=length(mmR), nc=length(mmC))
                
                ## map data from dataframe to matrix
                for(i in 1:nrow(pp)){
                    mm[which(mmR==pp[i,aXis[1]]),which(mmC==pp[i,aXis[2]])] <- pp[i,aXis[3]]
                };rm(i)
                "
                @rget mm mmR mmC
                if size(mm)[1] > 0
                    plot(mmR, mmC, mm, st=:surface,
                    xlabel=names(rAw)[aXis[1]], ylabel=names(rAw)[aXis[2]], zlabel=names(rAw)[aXis[3]],
                    title="Surface plot animation based on fixed parameters\n ePR="*string(fiX[1])*", eP="*string(fiX[2])*", eBR="*string(fiX[3])*", eB="*string(fiX[4])*", mB="*string(fiX[5])*",\n x="*string(u0[1][aMt0[2]])*", aP="*string(u0[aXis[4]][aMt0[1]]))
                end
            end
gif(aa, "gif/"*string(names(rAw)[aXis[3]])*".gif", fps=15)

In [ ]:
aXis[3] = 13 ## customed z axis
aa = @animate for i0 in 1:length(aMt)
                aMt0 = aMt[i0]
                @rput aXis aMt0
                R"
                ## extract necessary data
                pp = rAw[which(rAw[,aXis[4]]==u0[[aXis[4]]][aMt0[1]] & rAw[,1]==u0[[1]][aMt0[2]]),]
                
                ## initialize matrix
                mmR = unique(pp[,aXis[1]])
                mmC = unique(pp[,aXis[2]])
                mm = matrix(NA, nr=length(mmR), nc=length(mmC))
                
                ## map data from dataframe to matrix
                for(i in 1:nrow(pp)){
                    mm[which(mmR==pp[i,aXis[1]]),which(mmC==pp[i,aXis[2]])] <- pp[i,aXis[3]]
                };rm(i)
                "
                @rget mm mmR mmC
                if size(mm)[1] > 0
                    plot(mmR, mmC, mm, st=:surface,
                    xlabel=names(rAw)[aXis[1]], ylabel=names(rAw)[aXis[2]], zlabel=names(rAw)[aXis[3]],
                    title="Surface plot animation based on fixed parameters\n ePR="*string(fiX[1])*", eP="*string(fiX[2])*", eBR="*string(fiX[3])*", eB="*string(fiX[4])*", mB="*string(fiX[5])*",\n x="*string(u0[1][aMt0[2]])*", aP="*string(u0[aXis[4]][aMt0[1]]))
                end
            end
gif(aa, "gif/"*string(names(rAw)[aXis[3]])*".gif", fps=15)

In [ ]:
aXis[3] = 13 ## customed z axis
aa = @animate for i0 in 1:length(aMt)
                aMt0 = aMt[i0]
                @rput aXis aMt0
                R"
                ## extract necessary data
                pp = rAw[which(rAw[,aXis[4]]==u0[[aXis[4]]][aMt0[1]] & rAw[,1]==u0[[1]][aMt0[2]]),]
                
                ## initialize matrix
                mmR = unique(pp[,aXis[1]])
                mmC = unique(pp[,aXis[2]])
                mm = matrix(NA, nr=length(mmR), nc=length(mmC))
                
                ## map data from dataframe to matrix
                for(i in 1:nrow(pp)){
                    mm[which(mmR==pp[i,aXis[1]]),which(mmC==pp[i,aXis[2]])] <- pp[i,aXis[3]]
                };rm(i)
                "
                @rget mm mmR mmC
                if size(mm)[1] > 0
                    plot(mmR, mmC, mm, st=:surface,
                    xlabel=names(rAw)[aXis[1]], ylabel=names(rAw)[aXis[2]], zlabel=names(rAw)[aXis[3]],
                    title="Surface plot animation based on fixed parameters\n ePR="*string(fiX[1])*", eP="*string(fiX[2])*", eBR="*string(fiX[3])*", eB="*string(fiX[4])*", mB="*string(fiX[5])*",\n x="*string(u0[1][aMt0[2]])*", aP="*string(u0[aXis[4]][aMt0[1]]))
                end
            end
gif(aa, "gif/"*string(names(rAw)[aXis[3]])*".gif", fps=15)